In [2]:
# Dependencies

import pandas as pd 
from pathlib import Path
import requests
from pprint import pprint 

In [4]:
# Store filepath in a variable 

file_one = Path("Resources/us_counties_covid19_daily.csv")

In [5]:
# Read our data files with the Pandas library

covid_df = pd.read_csv(file_one, encoding= "ISO-8859-1")

In [6]:
# Show the first five rows

covid_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [8]:
# select rows in my season
# make a start date for december
# make a end date for december 

start_date = "2020-12-01"
end_date = "2020-12-31"


# make a start date for january-february 
# make a end date for january-febraury

start_date = "2020-01-01"
end_date = "2020-02-29"

